# Calendars Fundamentals

## Namespaces

You can use namespaces in your Python by importing the necessary modules.

In [1]:
# Add modules from standard library into our namespace
import datetime

# Add common models
from lseg_analytics.common import (
    DateMovingConvention,
    DayCountBasis,
    Description,
    Frequency,
    Month,
    PeriodType,
    ValidityPeriod,
    WeekDay,
)

# Add specific modules for calendars
from lseg_analytics.reference_data.calendars import (
    AbsolutePositionWhen,
    Calendar,
    CalendarDefinition,
    FullDayDuration,
    HolidayRule,
    RestDays,
    compute_dates,
    count_periods,
    delete,
    generate_date_schedule,
    generate_holidays,
    load,
    search,
)

You can import api modules using their original names:

In [2]:
from lseg_analytics.reference_data import calendars

calendars.Calendar

lseg_analytics.reference_data.calendars._calendar.Calendar

Or you can import them using an alias to prevent name conflicts:

In [3]:
from lseg_analytics.reference_data import calendars as cals

cals.Calendar, cals.load

(lseg_analytics.reference_data.calendars._calendar.Calendar,
 <function lseg_analytics.reference_data.calendars._functions.load(*, resource_id: Optional[str] = None, name: Optional[str] = None, space: Optional[str] = None)>)

## Constructors

Each object might have the following parameters:
- A definition containing information that *defines* a resource, used to build the resource object itself.
- A description containing information that *describes* a resource, for example a summary and/or tags.

The definition is mandatory while description is optionals.

### Definition

You can construct objects in more than one way. Define a calendar by giving the mandatory 'definition':

In [4]:
# Create a calendar instance with parameter.
my_cal_definition = CalendarDefinition(rest_days = [
                    RestDays(
                        rest_days=[WeekDay.SATURDAY, WeekDay.SUNDAY],
                        validity_period=ValidityPeriod(
                            start_date="2024-01-01",
                            end_date="2024-12-31",
                        ),
                    )
                ],
                    first_day_of_week = WeekDay.FRIDAY,
                    holiday_rules = [
                    HolidayRule(
                        name="New Year's Day",
                        duration=FullDayDuration(full_day=1),
                        validity_period=ValidityPeriod(
                            start_date="2024-01-01",
                            end_date="2024-12-31",
                        ),
                        when=AbsolutePositionWhen(day_of_month=1, month=Month.JANUARY),
                    ),
                ]
                )
my_cal = Calendar(definition=my_cal_definition)

### Description

You can also add a description later:

In [5]:
my_cal.description.summary = "My calendar"

my_cal.description.summary

'My calendar'

## Save

The example below creates a UK calendar that contains New Year's Day:

In [6]:
uk_cal = Calendar(
    definition=CalendarDefinition(
        rest_days=[
            RestDays(
                rest_days=[WeekDay.SATURDAY, WeekDay.SUNDAY],
                validity_period=ValidityPeriod(
                    start_date=datetime.date(2024, 1, 1),
                    end_date=datetime.date(2023, 12, 31),
                ),
            )
        ],
        first_day_of_week=WeekDay.MONDAY,
        holiday_rules=[
            HolidayRule(
                name="New Year's Day",
                duration=FullDayDuration(full_day=1),
                validity_period=ValidityPeriod(
                    start_date=datetime.date(2024, 1, 1),
                    end_date=datetime.date(2024, 12, 31),
                ),
                when=AbsolutePositionWhen(day_of_month=1, month=Month.JANUARY),
            ),
        ],
    ),
    description=Description(summary="United Kingdom calendar", tags=["UK"]),
)

uk_cal

<Calendar space=None name='' unsaved>

To save the calendar to a space, write:

In [7]:
uk_cal.save(name="UK_Calendar", space="HOME")
my_cal.save(name="My_Calendar", space="HOME")

True

## Location

Name and space are location attributes, which are automatically set when a resource object is saved for the first time.

Unsaved resources have thier name and space set to None.

Location attributes are read-only.

In [8]:
print(f"Calendar name: {uk_cal.location.name}")

print(f"Calendar space: {uk_cal.location.space}")

Calendar name: UK_Calendar
Calendar space: HOME


## Resource ID

A resource ID is the unique resource identifier for an object on the platform.

The resource ID is created on saving.

IDs are read-only

In [9]:
uk_cal_id = uk_cal.id

uk_cal_id

'80ea4c2e-6317-465c-a3f9-705878bac5cf'

## Load

Previously saved resourcess can be loaded in either by name or by ID.

### By name:

In [10]:
loaded_by_name_uk_cal = load(name="UK_Calendar")

### By ID:

In [11]:
uk_cal_loaded_by_id = load(resource_id=uk_cal_id)

Space is optional, but crucial if you need to have resources with the same name in different spaces:

In [12]:
uk_cal_loaded_by_name_and_space = load(name="UK_Calendar", space="HOME")

## Search

You can search for previously saved calendars using the following techniques.

### Search for all calendars

In [13]:
cals = search()

cals

[{'type': 'Calendar', 'id': '44703364-190d-4707-b29c-db0e6e261dc5', 'location': {'space': 'HOME', 'name': 'My_Calendar'}, 'description': {'summary': 'My calendar', 'tags': []}},
 {'type': 'Calendar', 'id': '3db1d034-a371-48ba-8ff7-c582d3bedade', 'location': {'space': 'HOME', 'name': 'my_personal_calendar'}, 'description': {'summary': '', 'tags': []}},
 {'type': 'Calendar', 'id': 'f4bab25f-70dd-4107-bf4e-82214e7004f0', 'location': {'space': 'HOME', 'name': 'TestCalendar1747794848181666'}, 'description': {'summary': '', 'tags': []}},
 {'type': 'Calendar', 'id': '80ea4c2e-6317-465c-a3f9-705878bac5cf', 'location': {'space': 'HOME', 'name': 'UK_Calendar'}, 'description': {'summary': 'United Kingdom calendar', 'tags': ['UK']}},
 {'type': 'Calendar', 'id': '6d253027-d629-42f4-9871-754ce8b02ff9', 'location': {'space': 'LSEG', 'name': 'ABW'}, 'description': {'summary': 'LSEG Aruba', 'tags': []}},
 {'type': 'Calendar', 'id': '8c5e56c2-ac93-4db1-bc89-9d0f4eb4bfa2', 'location': {'space': 'LSEG', '

### Search for calendars by name

In [14]:
cals_by_names = search(names=["EMU"])

cals_by_names

[{'type': 'Calendar', 'id': '0cee3640-8063-49b4-a1a1-50ab1ee0030f', 'location': {'space': 'LSEG', 'name': 'EMU'}, 'description': {'summary': 'LSEG Euroland Calendar', 'tags': []}}]

### Search for calendars by space

In [15]:
cals_by_spaces = search(spaces=["HOME"])

cals_by_spaces

[{'type': 'Calendar', 'id': '44703364-190d-4707-b29c-db0e6e261dc5', 'location': {'space': 'HOME', 'name': 'My_Calendar'}, 'description': {'summary': 'My calendar', 'tags': []}},
 {'type': 'Calendar', 'id': '3db1d034-a371-48ba-8ff7-c582d3bedade', 'location': {'space': 'HOME', 'name': 'my_personal_calendar'}, 'description': {'summary': '', 'tags': []}},
 {'type': 'Calendar', 'id': 'f4bab25f-70dd-4107-bf4e-82214e7004f0', 'location': {'space': 'HOME', 'name': 'TestCalendar1747794848181666'}, 'description': {'summary': '', 'tags': []}},
 {'type': 'Calendar', 'id': '80ea4c2e-6317-465c-a3f9-705878bac5cf', 'location': {'space': 'HOME', 'name': 'UK_Calendar'}, 'description': {'summary': 'United Kingdom calendar', 'tags': ['UK']}}]

### Search for calendars by tag

In [16]:
cals_by_tags = search(tags=["UK"])

cals_by_tags

[{'type': 'Calendar', 'id': '80ea4c2e-6317-465c-a3f9-705878bac5cf', 'location': {'space': 'HOME', 'name': 'UK_Calendar'}, 'description': {'summary': 'United Kingdom calendar', 'tags': ['UK']}}]

## Data access

You can access resource data through the root attributes of the resource object:

In [17]:
print(f"First day of week: {uk_cal.definition.first_day_of_week}")
print(
    f"Calendar holidays: {[holiday.name for holiday in uk_cal.definition.holiday_rules]}"
)

First day of week: WeekDay.MONDAY
Calendar holidays: ["New Year's Day"]


## Calendar functions

Calendar functions works with calendar instances.

In [18]:
ukg_cal = load(name="UKG")

### Count periods

Counts the time periods that satisfy specified conditions.

Note the use of date strings for convenience.

In [19]:
periods = ukg_cal.count_periods(
    start_date="2024-01-01",
    end_date="2024-12-31",
    day_count_basis=DayCountBasis.DCB_30_360_US,
    period_type=PeriodType.WORKING_DAY
)

periods

{'count': 253.0, 'periodType': 'WorkingDay'}

### Compute dates

Computes dates for the calendar according to specified conditions.

In [20]:
dates = ukg_cal.compute_dates(
    tenors=["1M", "2M"],
    start_date=datetime.date(2024, 1, 1),
    date_moving_convention=DateMovingConvention.NEXT_BUSINESS_DAY
)

dates

[{'tenor': '1M', 'endDate': '2024-02-01'},
 {'tenor': '2M', 'endDate': '2024-03-01'}]

### Generate date schedule

Generates a date schedule for the calendar according to specified conditions.

In [21]:
schedule = ukg_cal.generate_date_schedule(
    frequency=Frequency.WEEKLY,
    start_date=datetime.date(2024, 1, 1),
    count=5,
    day_of_week=WeekDay.THURSDAY
)

schedule

[datetime.date(2024, 1, 11),
 datetime.date(2024, 1, 18),
 datetime.date(2024, 1, 25),
 datetime.date(2024, 2, 1),
 datetime.date(2024, 2, 8)]

### Generate holidays

Gets the calendar's holidays.

In [22]:
holidays = ukg_cal.generate_holidays(
    start_date=datetime.date(2024, 1, 1),
    end_date=datetime.date(2024, 12, 31),
)

holidays

[{'date': '2024-01-01', 'names': [{'name': "New Year's Day", 'calendars': ['LSEG/UKG'], 'countries': ['GBR']}]},
 {'date': '2024-03-29', 'names': [{'name': 'Good Friday', 'calendars': ['LSEG/UKG'], 'countries': ['GBR']}]},
 {'date': '2024-03-31', 'names': [{'name': 'Easter Sunday', 'calendars': ['LSEG/UKG'], 'countries': ['GBR']}]},
 {'date': '2024-04-01', 'names': [{'name': 'Easter Monday', 'calendars': ['LSEG/UKG'], 'countries': ['GBR']}]},
 {'date': '2024-05-06', 'names': [{'name': 'Early May Bank Holiday', 'calendars': ['LSEG/UKG'], 'countries': ['GBR']}]},
 {'date': '2024-05-27', 'names': [{'name': 'Spring Bank Holiday', 'calendars': ['LSEG/UKG'], 'countries': ['GBR']}]},
 {'date': '2024-08-26', 'names': [{'name': 'Summer Bank Holiday', 'calendars': ['LSEG/UKG'], 'countries': ['GBR']}]},
 {'date': '2024-12-25', 'names': [{'name': 'Christmas Day', 'calendars': ['LSEG/UKG'], 'countries': ['GBR']}]},
 {'date': '2024-12-26', 'names': [{'name': 'Boxing Day', 'calendars': ['LSEG/UKG'], 

## Calendar module functions

Calendar module functions are similar to calendar functions, but work with multiple calendar instances.

### Count periods

Counts time periods that satisfy specified conditions for the calendar.

In [23]:
counted_periods = count_periods(
    start_date=datetime.date(2023, 1, 1),
    end_date=datetime.date(2023, 12, 31),
    day_count_basis=DayCountBasis.DCB_30_360,
    period_type=PeriodType.WORKING_DAY,
    calendars=[ukg_cal.id]
)

counted_periods

{'count': 251.0, 'periodType': 'WorkingDay'}

### Compute dates

Computes dates for the calendar according to specified conditions.

In [24]:
computed_dates = compute_dates(
    tenors=["1M", "2M"],
    calendars=[ukg_cal.id],
    start_date=datetime.date(2024, 1, 1),
    date_moving_convention=DateMovingConvention.NEXT_BUSINESS_DAY
)

computed_dates

[{'tenor': '1M', 'endDate': '2024-02-01'},
 {'tenor': '2M', 'endDate': '2024-03-01'}]

### Generate date schedule

Generates a schedule of dates for the calendar according to specified conditions.

In [25]:
generated_date_schedule = generate_date_schedule(
    frequency=Frequency.WEEKLY,
    calendars=[ukg_cal.id],
    start_date=datetime.date(2023, 1, 1),
    count=7,
    day_of_week=WeekDay.TUESDAY
)

generated_date_schedule

[datetime.date(2023, 1, 10),
 datetime.date(2023, 1, 17),
 datetime.date(2023, 1, 24),
 datetime.date(2023, 1, 31),
 datetime.date(2023, 2, 7),
 datetime.date(2023, 2, 14),
 datetime.date(2023, 2, 21)]

### Generate holidays

Gets the holidays for the calendar within a date range.

In [26]:
generated_holidays = generate_holidays(
    calendars=[ukg_cal.id],
    start_date=datetime.date(2024, 1, 1),
    end_date=datetime.date(2024, 12, 31),
)

generated_holidays

[{'date': '2024-01-01', 'names': [{'name': "New Year's Day", 'calendars': ['LSEG/UKG'], 'countries': ['GBR']}]},
 {'date': '2024-03-29', 'names': [{'name': 'Good Friday', 'calendars': ['LSEG/UKG'], 'countries': ['GBR']}]},
 {'date': '2024-03-31', 'names': [{'name': 'Easter Sunday', 'calendars': ['LSEG/UKG'], 'countries': ['GBR']}]},
 {'date': '2024-04-01', 'names': [{'name': 'Easter Monday', 'calendars': ['LSEG/UKG'], 'countries': ['GBR']}]},
 {'date': '2024-05-06', 'names': [{'name': 'Early May Bank Holiday', 'calendars': ['LSEG/UKG'], 'countries': ['GBR']}]},
 {'date': '2024-05-27', 'names': [{'name': 'Spring Bank Holiday', 'calendars': ['LSEG/UKG'], 'countries': ['GBR']}]},
 {'date': '2024-08-26', 'names': [{'name': 'Summer Bank Holiday', 'calendars': ['LSEG/UKG'], 'countries': ['GBR']}]},
 {'date': '2024-12-25', 'names': [{'name': 'Christmas Day', 'calendars': ['LSEG/UKG'], 'countries': ['GBR']}]},
 {'date': '2024-12-26', 'names': [{'name': 'Boxing Day', 'calendars': ['LSEG/UKG'], 

## Delete

You can delete a calendar from the server in the following ways:

### By ID:

In [27]:
delete(resource_id=uk_cal_id)

True

### By name and space unique pair

In [28]:
delete(name="My_Calendar", space="HOME")

True